### Add Satellite Remote Sensing Data

In [ ]:
import ee
import geemap

In [ ]:
ee.Initialize()

In [ ]:
Map = geemap.Map()

### Sentinel-2 

In [ ]:
#*
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2clouds(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [ ]:
dataset = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2020-02-01', '2020-06-30') \
                                                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                                                .map(maskS2clouds)

In [ ]:
visualization = {'min': 0.0,
                 'max': 0.3,
                 'bands': ['B4', 'B3', 'B2'],
                }

In [ ]:
#example in Lake of Tempe,South of Sulawesi
Map.setCenter(120.007486,-4.130420, 12)
Map.addLayer(dataset.mean(), visualization, 'RGB')
Map

### Sentinel-1

In [ ]:
# Load the Sentinel-1 ImageCollection.
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

In [ ]:
# Filter by metadata properties.
vh = sentinel1 \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

In [ ]:
# Filter to get images from different look angles.
vhAscending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

In [ ]:
# Create a composite from means at different polarizations and look angles.
composite = ee.Image.cat([vhAscending.select('VH').mean(),
                          ee.ImageCollection(vhAscending.select('VV').
                                             merge(vhDescending.select('VV'))).mean(),
                          vhDescending.select('VH').mean()
                         ]).focal_median()

In [ ]:
# Display as a composite of polarization and backscattering characteristics.
Map.setCenter(120.007486,-4.130420, 12)
Map.addLayer(composite, {'min': [-25,  -20,  -25], 'max': [0, 10, 0]}, 'composite')
Map